<a href="https://colab.research.google.com/github/Decoding-Data-Science/airesidency/blob/main/crewai_15thfeb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [4]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [5]:
import os

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [42]:
import os
from google.colab import userdata

# Retrieve API key from Colab secrets and set it in environment variables
os.environ["OPENAI_API_KEY"] = userdata.get('openai')

In [27]:
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

In [37]:
# Set up API keys - enter from https://serper.dev/billing
os.environ["SERPER_API_KEY"] = "" # serper.dev API key

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

In [29]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic."    ,
    tools=[search_tool, web_rag_tool,docs_tool,file_tool],
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [30]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic in 1500 words: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [31]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [32]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [33]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.1500 words",
    agent=writer,
)

### Task: Edit

In [34]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [35]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    planning=True,
    verbose=4
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [43]:
result = crew.kickoff(inputs={"topic": "who won 2025 Delhi elections"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on who won 2025 Delhi elections.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I need to gather information on the latest trends, key players, and noteworthy news regarding the 2025 Delhi elections to create a comprehensive content plan.

Action: Search the internet
Action Input: {"search_query": "2025 Delhi elections latest trends and key players"} 

{'message': 'Unauthorized.', 'statusCode': 403}

Action: Search in a specific website
Action Input: {"search_query": "2025 Delhi elections latest trends and key players", "website": "https://www.delhielections2025.com"} 

Relevant Content:


Thought: I need to gather information 

- Display the results of your execution as markdown in the notebook.

In [39]:
from IPython.display import Markdown
Markdown(result)

# Who Won the 2025 Delhi Elections? Analysis of Key Players, Trends, and Results

## Introduction
The 2025 Delhi Legislative Assembly elections garnered significant attention from political enthusiasts and voters eagerly awaiting the results. The outcome of these elections held immense importance for the future of Delhi politics, ultimately shaping the direction of governance in the region. As the dust settled and the results were announced, the focus shifted to analyzing the key players, trends, and overall impact of the election outcome.

## Key Players
In the lead-up to the 2025 Delhi elections, the Bharatiya Janata Party (BJP) and the Aam Aadmi Party (AAP) emerged as the major contenders, with their respective leaders Prime Minister Narendra Modi and Chief Minister Arvind Kejriwal vying for the top spot. The BJP emphasized development and governance in their campaign, while the AAP highlighted their achievements in education and healthcare. The presence of these key players intensified the electoral battle, capturing the attention of voters across Delhi.

## Latest Trends
Throughout the election campaign, several key trends emerged, providing insights into the electorate's pulse. The emphasis on local issues like infrastructure development and pollution control resonated with voters, reflecting their priorities. The utilization of social media and digital platforms played a crucial role in reaching a broader audience, influencing voter opinions. The mobilization of youth and women voters indicated a shift in traditional voting patterns, showcasing the evolving dynamics of Delhi politics.

## Results
The 2025 Delhi election results showcased the BJP's victory, securing a majority and significantly impacting Delhi politics. While the AAP put up a tough fight, they faced defeat in some constituencies, resulting in a mixed outcome. The winners of the election demonstrated their potential to influence the future governance of Delhi, setting the stage for policy decisions and legislative actions.

## Target Audience Analysis
This analysis targets individuals interested in Indian politics, specifically focusing on the Delhi region. Voters, political enthusiasts, and journalists seeking detailed insights into the 2025 Delhi elections form a significant part of the audience. Their interest lies in comprehending the election outcome, its implications, and the key players shaping Delhi's political landscape. Providing accurate information is essential for engaging this audience and offering valuable perspectives on the election results.

## SEO Keywords
To optimize the content for search engines, this analysis incorporates relevant keywords related to the 2025 Delhi elections. Keywords such as "2025 Delhi elections," "Delhi Assembly election results," "BJP wins Delhi elections," "AAP performance in Delhi polls," and "key players in Delhi politics" are strategically integrated. By leveraging these keywords effectively, the analysis aims to reach a broader audience interested in the Delhi elections and provide them with valuable insights.

## Resources
For further reading and in-depth analysis of the 2025 Delhi elections, credible sources like Wikipedia, BBC News, Election Commission of India, Times of India, India Today, NDTV, The Hindu, AP News, and Indian Express offer comprehensive coverage. These resources provide a detailed overview of the election trends, results, and key highlights, enriching the analysis and offering a holistic perspective on the 2025 Delhi elections.

In conclusion, the 2025 Delhi elections marked a significant political showdown, with the BJP emerging victorious and shaping the future trajectory of Delhi politics. The analysis of key players, trends, and results offers valuable insights into the electoral dynamics and governance implications in the region. By engaging the target audience and incorporating SEO best practices, this analysis provides a comprehensive overview of the 2025 Delhi elections, catering to the diverse needs of readers.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "AI Legislation"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on AI Legislation.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Title: Understanding AI Legislation: A Comprehensive Guide to the Latest Trends and Key Players

Introduction:
- Brief overview of the increasing importance of AI legislation in today's digital world.
- Introduction to the purpose of the article: to provide insights into the latest trends, key players, and noteworthy news in AI legislation.

Key Points:
1. Latest Trends in AI Legislation:
- Discussion on the increasing global focus on regulating AI technology.
- Overview of recent AI legislation developments in major c

In [ ]:
from IPython.display import Markdown
Markdown(result)

# Understanding AI Legislation: A Comprehensive Guide to the Latest Trends and Key Players

## Introduction
In today's rapidly evolving digital landscape, the importance of AI legislation cannot be overstated. As artificial intelligence continues to permeate various aspects of our lives, regulations governing its use are becoming increasingly crucial. This article aims to delve into the latest trends, key players, and noteworthy news in AI legislation, providing a comprehensive guide for those navigating this complex terrain.

## Latest Trends in AI Legislation
The global focus on regulating AI technology has been steadily increasing as policymakers grapple with the ethical, legal, and societal implications of its widespread adoption. Major countries such as the US, EU, and China have been at the forefront of developing AI legislation, with a keen eye on industries like healthcare, finance, and transportation. The impact of AI legislation on these sectors is profound, shaping the way businesses operate and individuals interact with emerging technologies.

## Key Players in AI Legislation
Key governmental bodies and organizations play a pivotal role in shaping AI legislation, working alongside influential policymakers, industry leaders, and advocacy groups. These stakeholders collaborate to establish frameworks that balance innovation with accountability and transparency. Case studies highlighting successful implementation of AI legislation showcase the positive outcomes of proactive regulatory measures, underscoring the importance of collaboration in this dynamic space.

## Noteworthy News in AI Legislation
Recent news articles, reports, and studies shed light on the evolving landscape of AI legislation, sparking debates and discussions on controversial topics. As upcoming events and conferences focus on AI legislation, the opportunity to engage with thought leaders and experts in the field presents itself. By staying informed and actively participating in these dialogues, individuals can contribute to shaping the future of AI governance and policy.

## Conclusion
In conclusion, understanding AI legislation is essential for navigating the complexities of a digital world powered by artificial intelligence. By keeping abreast of the latest trends, engaging with key players, and staying informed on noteworthy news, individuals can position themselves as active participants in shaping the future of AI regulation. I encourage readers to explore the resources provided and join the conversation on AI legislation, as we collectively strive towards a more ethical, transparent, and inclusive AI ecosystem.

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).